In [18]:
import os, shutil, pathlib
from tensorflow import keras
from keras import layers
import numpy as np
from keras.utils import image_dataset_from_directory

In [5]:
original_dir = pathlib.Path('C:/Users/KSY/Desktop/Codes/AI/Ch_08/microsoft-catsvsdogs-dataset/PetImages/')
new_base_dir = pathlib.Path('cats_vs_dogs_small')

In [8]:
# start_index에서 end_index까지의 고양이와 강아지 이미지를 new_base_dir/{subset_name}/cat(dog)으로 복사
# 유틸리티 함수 subset_name은 train, validation, test 중 하나
def make_subset(subset_name, start_index, end_index) : 
    for category in ('cat', 'dog') :
        dir = new_base_dir / subset_name / category 
        os.makedirs(dir)
        fnames = [f"{i}.jpg" #0.jpg와 같이 저장되어 있음
                  for i in range(start_index, end_index)]
        for fname in fnames :
            shutil.copyfile(src=original_dir / category / fname, dst=dir / fname) # 중간에 category가 포함

In [9]:
make_subset("train", start_index=0, end_index=1000)

In [10]:
make_subset("validation", start_index=1000, end_index=1500)

In [11]:
make_subset('test', start_index=1500, end_index=2500)

In [17]:
inputs = keras.Input(shape=(180, 180, 3)) # 180 * 180 * 3 크기의 이미지를 기대
x = layers.Rescaling(1./255)(inputs) # 입력을 255로 나누어 0 ~ 1 범위로 조정
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x) # 출력 크기 178, 178, 32
x = layers.MaxPooling2D(pool_size=2)(x) # 출력 크기 89 * 89 * 32
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x) # 출력 크기 87, 87, 64
x = layers.MaxPooling2D(pool_size=2)(x) # 출력 크기 43, 43, 64
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x) # 출력 크기 41, 41, 128
x = layers.MaxPooling2D(pool_size=2)(x) # 출력 크기 20, 20, 128
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x) # 출력 크기 18, 18, 256
x = layers.MaxPooling2D(pool_size=2)(x) # 출력 크기 9, 9, 256
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x) # 출력 크기 7, 7, 256
x = layers.Flatten()(x) # 출력 크기 7 * 7 * 256 = 12,545
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [19]:
train_dataset = image_dataset_from_directory(
    new_base_dir / "train", # 아래 있는 디렉터리를 class로 가정
    image_size=(180, 180),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation", # 아래 있는 디렉터리를 class로 가정
    image_size=(180, 180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test", # 아래 있는 디렉터리를 class로 가정
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [20]:
for data_batch, labels_batch in train_dataset :
    print("데이터 배치 크기:", data_batch.shape)
    print("레이블 배치 크기:", labels_batch.shape)
    break

데이터 배치 크기: (32, 180, 180, 3)
레이블 배치 크기: (32,)
